In [1]:
import requests
from bs4 import BeautifulSoup

/home/husein/.local/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.15) or chardet (5.2.0)/charset_normalizer (2.0.7) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [2]:
r = requests.get('https://www.mjpath.org.my/past-issue.php')

In [4]:
soup = BeautifulSoup(r.content)
a = soup.find_all('a')
pages = ['https://www.mjpath.org.my/' + a_.get('href') for a_ in a if 'index.php' in a_.get('href')]
pages

['https://www.mjpath.org.my/index.php',
 'https://www.mjpath.org.my/index.php',
 'https://www.mjpath.org.my/index.php',
 'https://www.mjpath.org.my/2022/v44n3/index.php',
 'https://www.mjpath.org.my/2023/v45n1/index.php',
 'https://www.mjpath.org.my/2022/v44n2/index.php',
 'https://www.mjpath.org.my/2022/v44n1/index.php',
 'https://www.mjpath.org.my/2021/v43n3/index.php',
 'https://www.mjpath.org.my/2021/v43n2/index.php',
 'https://www.mjpath.org.my/2020/v42n3/index.php',
 'https://www.mjpath.org.my/2021/v43n1/index.php',
 'https://www.mjpath.org.my/2020/v42n1/index.php',
 'https://www.mjpath.org.my/2020/v42n2/index.php',
 'https://www.mjpath.org.my/2019/v41n3/index.php',
 'https://www.mjpath.org.my/2019/v41n2/index.php',
 'https://www.mjpath.org.my/2018/v40n3/index.php',
 'https://www.mjpath.org.my/2019/v41n1/index.php',
 'https://www.mjpath.org.my/2018/v40n1/index.php',
 'https://www.mjpath.org.my/2018/v40n2/index.php',
 'https://www.mjpath.org.my/2017/v39n3/index.php',
 'https://www

In [7]:
from tqdm import tqdm

urls = []
for p in tqdm(pages):
    r = requests.get(p)
    soup = BeautifulSoup(r.content)
    a = soup.find_all('a')
    a = [(p, a_.get('href')) for a_ in a if a_.get('href').endswith('.pdf')]
    urls.extend(a)
    
urls

100%|███████████████████████████████████████████| 92/92 [00:41<00:00,  2.19it/s]


[('https://www.mjpath.org.my/index.php', '2023/v45n2/contents.pdf'),
 ('https://www.mjpath.org.my/index.php', '2023/v45n2/prostate-cancer.pdf'),
 ('https://www.mjpath.org.my/index.php',
  '2023/v45n2/anti-DFS70-antibodies.pdf'),
 ('https://www.mjpath.org.my/index.php', '2023/v45n2/breast-carcinoma.pdf'),
 ('https://www.mjpath.org.my/index.php',
  '2023/v45n2/transfusion-medicine-knowledge.pdf'),
 ('https://www.mjpath.org.my/index.php', '2023/v45n2/exosomes.pdf'),
 ('https://www.mjpath.org.my/index.php',
  '2023/v45n2/acute-myeloid-leukaemia.pdf'),
 ('https://www.mjpath.org.my/index.php',
  '2023/v45n2/respiratory-infection.pdf'),
 ('https://www.mjpath.org.my/index.php',
  '2023/v45n2/ovarian-surface-epithelial-tumours.pdf'),
 ('https://www.mjpath.org.my/index.php', '2023/v45n2/oral-candidiasis.pdf'),
 ('https://www.mjpath.org.my/index.php',
  '2023/v45n2/mesenchymal-stem-cells.pdf'),
 ('https://www.mjpath.org.my/index.php',
  '2023/v45n2/urothelial-carcinoma.pdf'),
 ('https://www.mjpat

In [8]:
urls_combined = []
for u in urls:
    u_ = u[0].split('/index.php')[0]
    urls_combined.append(f'{u_}/{u[1]}')

In [11]:
urls_combined = sorted(list(set(urls_combined)))
len(urls_combined)

1351

In [12]:
!mkdir mjpath.org.my

In [13]:
import os

pdfs = urls_combined
for i in tqdm(range(len(pdfs))):
    filename = os.path.join('mjpath.org.my', f'{i}.pdf')
    if os.path.exists(filename):
        continue
        
    u = pdfs[i]
        
    r = requests.get(u)
    with open(filename, 'wb') as fopen:
        fopen.write(r.content)

100%|███████████████████████████████████████| 1351/1351 [19:59<00:00,  1.13it/s]


In [14]:
from glob import glob
from tika import parser
import json

TIKA_HOST = 'http://localhost:9998'

In [15]:
files = glob('mjpath.org.my/*.pdf')

with open('mjpath.org.my.jsonl', 'w') as fopen:
    for f in tqdm(files):
        try:
            raw_xml = parser.from_file(f, TIKA_HOST, xmlContent=True)
            body = BeautifulSoup(raw_xml['content']).find('body')
            t = '\n'.join(body.findAll(text=True))
            fopen.write(f'{json.dumps(t)}\n')
        except:
            pass

100%|███████████████████████████████████████| 1351/1351 [00:29<00:00, 45.76it/s]
